# Assembly and analysis of *V. dentatum* RAD data set

A library for 84 samples was prepared by Floragenex with the PstI enzyme. The library was sequenced on xx lanes of an Illumina HiSeq 2500 yielding xxx reads.

### This notebook
This notebook provides a fully reproducible workflow to assemble and analyze the Spriggs et al. *V. dentatum* RAD data set. Starting from the demultiplexed data files, we denovo assemble the data in *ipyrad*, which involves filtering reads,  clustering within and between samples to identify homology, and final filtering and formating to create output files. 

In [1]:
## all necessary software can be installed by uncommenting the command below
# conda install -c ipyrad ipyrad -y

### Import Python modules

In [2]:
## import basic modules and ipyrad
import os
import socket
import glob
import subprocess as sps
import numpy as np
import ipyparallel as ipp
import ipyrad as ip

## print versions 
print "ipyrad v.{}".format(ip.__version__)
print "ipyparallel v.{}".format(ipp.__version__)
print "numpy v.{}".format(np.__version__)

ipyrad v.0.5.13
ipyparallel v.5.2.0
numpy v.1.11.2


### The cluster
This notebook is connected to multiple cores on the Farnam HPC cluster at Yale. SSH Tunneling was set up following [this tutorial](http://ipyrad.readthedocs.io/HPC_Tunnel.html) to launch an *ipcluster* instance connected to all nodes. Below I use the ipyparallel Python module to show the cores that are connected and which ipyrad will make use of. 

In [149]:
## open a view to the client
ipyclient = ipp.Client(cluster_id="ip-8899")
lbview = ipyclient.load_balanced_view()

## confirm we are connected to 4 8-core nodes
hosts = ipyclient[:].apply_sync(socket.gethostname)
for hostname in set(hosts):
    print("host compute node: [{} cores] on {}"\
          .format(hosts.count(hostname), hostname))

host compute node: [20 cores] on c13n02.farnam.hpc.yale.internal
host compute node: [20 cores] on c13n01.farnam.hpc.yale.internal


In [152]:
base = ip.Assembly('test', quiet=True)
base._ipcluster["cluster_id"] = "ip-8899"
base._ipcluster["cores"] = 40
base.set_params('sorted_fastq_path', lib1)
base.set_params('project_dir', WORK)
base.run("1")


  Assembly: test
  [####################] 100%  loading reads         | 0:00:14 | s1 | 


In [ ]:
## the above MPI code works even without importing mpi4py
#%px import mpi4py as MPI; MPI.is_Initalized()

### Set up a working directory

In [153]:
## create a new working directory in HPC scratch dir
WORK = "/ysm-gpfs/scratch60/de243/nudum-ELS-RAD"
if not os.path.exists(WORK):
    os.mkdir(WORK)

## the current dir (./) in which this notebook resides
NBDIR = os.path.realpath(os.curdir)

## print both
print "working directory (WORK) = {}".format(WORK)
print "current directory (NBDIR) = {}".format(NBDIR)

working directory (WORK) = /ysm-gpfs/scratch60/de243/nudum-ELS-RAD
current directory (NBDIR) = /ysm-gpfs/home/de243/Viburnum-ELS-RAD


### Load in demultiplexed data files
The raw R1 data were previously demultiplexed from libraries that include many other species of *Viburnum*. The demux fastq files are temporarily stored in the scratch directory below.

In [154]:
## Locations of the demux files stored temporarily on Yale's Farnam HPC cluster
## There are 8 locations that data could be from, let's just read in all in
## and then pick and choose samples from it. 
lib1 = "/ysm-gpfs/home/de243/project/Viburnum_demux/lib1-lane1_fastqs/*.gz"
lib2 = "/ysm-gpfs/home/de243/project/Viburnum_demux/lib1-lane2_fastqs/*.gz"
lib3 = "/ysm-gpfs/home/de243/project/Viburnum_demux/lib2-lane1_fastqs/*.gz"
lib4 = "/ysm-gpfs/home/de243/project/Viburnum_demux/lib2-lane2_fastqs/*.gz"
lib5 = "/ysm-gpfs/home/de243/project/Viburnum_demux/lib3-lane1_fastqs/*.gz"
lib6 = "/ysm-gpfs/home/de243/project/Viburnum_demux/lib3-lane2_fastqs/*.gz"
lib7 = "/ysm-gpfs/home/de243/project/Viburnum_demux/lib4-lane1_fastqs/*.gz"
lib8 = "/ysm-gpfs/home/de243/project/Viburnum_demux/lib4-lane2_fastqs/*.gz"

In [155]:
## make a list of the data locations
libs = [lib1, lib2, lib3, lib4, lib5, lib6, lib7, lib8]

## a dictionary to store assemblies in
adict = {}

## iterate over the list creating assemblies and setting data locations
for idx, path in enumerate(libs): 
    ## a name for this assembly
    name = "lib" + str(idx+1)
    
    ## create an Assembly of the 
    adict[name] = ip.Assembly(name, quiet=True)
    adict[name]._ipcluster["cluster_id"] = "ip-8899"
    
    ## set the sorted_fastq_path
    adict[name].set_params('sorted_fastq_path', path)
    adict[name].set_params('project_dir', WORK)
    
    ## load in the data
    adict[name].run("1")


  Assembly: lib1
  [####################] 100%  loading reads         | 0:00:14 | s1 | 

  Assembly: lib2
  [####################] 100%  loading reads         | 0:00:15 | s1 | 

  Assembly: lib3
  [####################] 100%  loading reads         | 0:00:11 | s1 | 

  Assembly: lib4
  [####################] 100%  loading reads         | 0:00:08 | s1 | 

  Assembly: lib5
  [####################] 100%  loading reads         | 0:00:11 | s1 | 

  Assembly: lib6
  [####################] 100%  loading reads         | 0:00:11 | s1 | 

  Assembly: lib7
  [####################] 100%  loading reads         | 0:00:09 | s1 | 

  Assembly: lib8
  [####################] 100%  loading reads         | 0:00:09 | s1 | 


### Merge all of the assemblies together
This joins all of the Samples into one Assembly, and joins multiple demultiplexed fastq files into a single Sample when it has multiple lanes of data, based on there being multiple individuals with identical names. 

In [156]:
## the second argument to merge() is a list of assemblies to merge
data = ip.merge("all-merged", adict.values())

## print a summary of the number of reads per sample 
print data.stats.describe().astype(int)

       state  reads_raw
count    381        381
mean       1    2606079
std        0    3180752
min        1     277809
25%        1    1660163
50%        1    2186937
75%        1    2878413
max        1   59023768


### Select only the subset of data that will be used in this study
For the *Viburnum nudum*-complex we will include samples of *cassinoides, nudum, rufidulum, prunifolium, punctatum, elatum, obovatum, lentago,* and several outgroups. We can select these

In [157]:
## get a list of names of all Samples
allnames = data.samples.keys()

## get a list of names of focal taxa to keep including a few outgroups
keeptaxa = ["cassinoides", "nudum",
            "carlesii", "utile", "macrocephalum", 
            "plicatum", "lutescens", "erubescens", "lantanoides"]

keeptaxa.append()
#"rufidulum", "prunifolium",
#"punctatum", "elatum", "obovatum", "lentago", 

## a list of Sample names to keep. We will populate this list below.
keepsamples = []

## get list of Sample names that have the keeptaxa names in them
for name in allnames:
    ## if name does not have one of our keeptaxa names in it
    if any([i in name for i in keeptaxa]):
        ## remove name from list to keep
        keepsamples.append(name)

## one extra sample with a similar name snuck in that we want to exclude
keepsamples.remove("PWS2265_leiocarpum_punctatum")

### Create a new Assembly with only the subsampled Samples

In [158]:
## we'll name the new Assembly 'base'
base = data.branch("base", subsamples=keepsamples)

## print a summary of stats
print base.stats.describe().astype(int)

       state  reads_raw
count    162        162
mean       1    2291171
std        0    1199120
min        1     277809
25%        1    1545788
50%        1    2088650
75%        1    2820438
max        1    9559578


### Start an new assembly using all data
We start from the demultiplexed data files that dat1 and dat2 are linked to. To combine them into a single assembly we use the 'merge' command below. This new assembly, which we will refer to as 'data' will be the main object we use to assemble the data through steps 2-6. We will therefore set all of the assembly parameters on this Assembly.

In [159]:
## set parameters for this assembly and print them 
base.set_params("project_dir", os.path.join(WORK, base.name))
base.set_params("filter_adapters", 2)
base.set_params("max_Hs_consens", (5, 5))
base.set_params("trim_overhang", (0, 5, 0, 0))
base.get_params()

  0   assembly_name               base                                         
  1   project_dir                 /ysm-gpfs/scratch60/de243/nudum-ELS-RAD/base 
  2   raw_fastq_path                                                           
  3   barcodes_path                                                            
  4   sorted_fastq_path           /ysm-gpfs/home/de243/project/Viburnum_demux/lib4-lane2_fastqs/*.gz
  5   assembly_method             denovo                                       
  6   reference_sequence                                                       
  7   datatype                    rad                                          
  8   restriction_overhang        ('TGCAG', '')                                
  9   max_low_qual_bases          5                                            
  10  phred_Qscore_offset         33                                           
  11  mindepth_statistical        6                                            
  12  mindepth_majr

### Run step2 filtering 

In [160]:
## the common Illumina adapter
print base._hackersonly["p3_adapter"]

## add poly-A and poly-T repeats to the list of things to filter out
base._hackersonly["p3_adapters_extra"] = ["TTTTTTTTTT", "AAAAAAAAAA"]

## print our adapter and seqs to be trimmed
for seq in base._hackersonly["p3_adapters_extra"]:
    print seq

AGATCGGAAGAGC
TTTTTTTTTT
AAAAAAAAAA


In [161]:
## it will skip step1 and run step2
base.run("2", show_cluster=True)

  local compute node: [20 cores] on c13n01.farnam.hpc.yale.internal

  Assembly: base
  [####################] 100%  concatenating inputs  | 0:01:12 | s2 | 
  [####################] 100%  processing reads      | 0:11:57 | s2 | 


In [162]:
## print a summary of the results
print base.stats_dfs.s2#.describe().astype(int)

       reads_raw  trim_adapter_bp_read1  trim_quality_bp_read1  \
count        162                    162                    162   
mean     2291171                 448974                5989627   
std      1199120                 206058                6404216   
min       277809                 108107                 660327   
25%      1545788                 311663                2032786   
50%      2088650                 406223                3325117   
75%      2820438                 561390                6801718   
max      9559578                1471855               34283506   

       reads_filtered_by_Ns  reads_filtered_by_minlen  reads_passed_filter  
count                   162                       162                  162  
mean                    319                    226312              2064539  
std                     220                    152299              1188402  
min                      10                     46900                72863  
25%                 

### Assemble data sets under a range of params

In [ ]:
adict = {}

for clust in ["0.85", "0.88", "0.92"]:
    
    ## create branched assemblies
    name = "base-c{}".format(clust[2:])
    cbase = base.branch(name)
    cbase.set_params('clust_threshold', clust)
    cbase.run("3")
    
    for mindepth in [6, 20, 40]:
        
        ## create branched assemblies
        name = "base-c{}-d{}".format(clust[2:], mindepth)
        dbase = cbase.branch(name)
        dbase.set_params("mindepth_statistical", mindepth)
        dbase.set_params("mindepth_majrule", mindepth)
        dbase.run("456")
        
        for minsamp in [10, 20, 40]:
            
            ## create final branched assemblies
            name = "nudum-c{}-d{}-min{}".format(clust[2:], mindepth, minsamp)
            sbase = dbase.branch(name)
            sbase.set_params("min_samples_locus", minsamp)
            sbase.run("7")
            
            ## save assembly to dict
            adict[sbase.name] = sbase          
        


  Assembly: base-c85
  [####################] 100%  dereplicating         | 0:00:00 | s3 | 
  [####################] 100%  clustering            | 0:16:31 | s3 | 
  [####################] 100%  building clusters     | 0:00:40 | s3 | 
  [####################] 100%  chunking              | 0:00:06 | s3 | 
  [###                 ]  15%  aligning              | 3:04:19 | s3 | 

#### Cluster reads within and across samples (steps 3-6)

In [19]:
## optional: optimize cluster threading with vsearch
data._ipcluster["threads"] = 4


In [22]:
data.run("3456")


  Assembly: c90d6f2h5
  [####################] 100%  dereplicating         | 0:01:19 | s3 | 
  [####################] 100%  clustering            | 1:36:44 | s3 | 
  [####################] 100%  building clusters     | 0:00:41 | s3 | 
  [####################] 100%  chunking              | 0:00:02 | s3 | 
  [####################] 100%  aligning              | 4:04:12 | s3 | 
  [####################] 100%  concatenating         | 0:00:56 | s3 | 
  [####################] 100%  inferring [H, E]      | 2:38:43 | s4 | 
  [####################] 100%  calculating depths    | 0:00:58 | s5 | 
  [####################] 100%  chunking clusters     | 0:01:46 | s5 | 
  [####################] 100%  consens calling       | 1:47:28 | s5 | 
  [####################] 100%  concat/shuffle input  | 0:01:23 | s6 | 
  [####################] 100%  clustering across     | 4:56:17 | s6 | 
  [####################] 100%  building clusters     | 0:01:11 | s6 | 
  [####################] 100%  aligning clusters     |

In [15]:
data.run("3456")


  Assembly: c90d6f2h5
  [####################] 100%  dereplicating         | 0:00:00 | s3 | 
  [####################] 100%  clustering            | 0:57:02 | s3 | 
  [####################] 100%  building clusters     | 0:00:38 | s3 | 
  [####################] 100%  chunking              | 0:00:03 | s3 | 
  [####################] 100%  aligning              | 1:47:36 | s3 | 
  [####################] 100%  concatenating         | 0:00:20 | s3 | 
  [####################] 100%  inferring [H, E]      | 1:32:41 | s4 | 
  [####################] 100%  consensus calling     | 0:54:10 | s5 | 
  [####################] 100%  concat/shuffle input  | 0:01:57 | s6 | 
  [####################] 100%  clustering across     | 6:00:59 | s6 | 
  [####################] 100%  building clusters     | 0:01:34 | s6 | 
  [####################] 100%  aligning clusters     | 0:14:03 | s6 | 
  [####################] 100%  database indels       | 0:11:51 | s6 | 
  [####################] 100%  indexing clusters     |

### Create final branches with different min sample values

In [23]:
## create named branches for final assemblies
min4 = data.branch("min4_c90d6f2h5")
min4.set_params("min_samples_locus", 4)

min10 = data.branch("min10_c90d6f2h5")
min10.set_params("min_samples_locus", 10)

## assemble outfiles
min4.run("7", force=True)
min10.run("7", force=True)


  Assembly: min4_c90d6f2h5
  [####################] 100%  filtering loci        | 0:01:30 | s7 | 
  [####################] 100%  building loci/stats   | 0:00:22 | s7 | 
  [####################] 100%  building vcf file     | 0:03:57 | s7 | 
  [####################] 100%  writing vcf file      | 0:00:02 | s7 | 
  [####################] 100%  writing outfiles      | 0:01:24 | s7 | 
  Outfiles written to: /ysm-gpfs/scratch60/de243/dentatum-ELS-RAD/c90d6f2h5/min4_c90d6f2h5_outfiles

  Assembly: min10_c90d6f2h5
  [####################] 100%  filtering loci        | 0:00:42 | s7 | 
  [####################] 100%  building loci/stats   | 0:00:19 | s7 | 
  [####################] 100%  building vcf file     | 0:02:39 | s7 | 
  [####################] 100%  writing vcf file      | 0:00:01 | s7 | 
  [####################] 100%  writing outfiles      | 0:00:47 | s7 | 
  Outfiles written to: /ysm-gpfs/scratch60/de243/dentatum-ELS-RAD/c90d6f2h5/min10_c90d6f2h5_outfiles


### Print some final results

In [25]:
print min4.stats

                          state  reads_raw  reads_passed_filter  \
BP001_dentatum                6    6762951              6406703   
ELS161_dentatum               6    1604990              1375652   
ELS163_dentatum               6    2636129              2489458   
ELS170_dentatum               6    2102752              1940042   
ELS193_dentatum               6    1569486              1362046   
ELS203_dentatum               6    2277428              2012833   
ELS208_dentatum               6     831337               553954   
ELS213_dentatum               6    1144773               875305   
ELS216_dentatum               6    2068878              1869347   
ELS272_dentatum               6    2563078              2254325   
ELS275_dentatum               6    2063939              1695440   
ELS281_dentatum               6    2682786              2487089   
ELS291_dentatum               6    2312460              2123516   
ELS292_dentatum               6    2444413              236581

In [26]:
!cat $min4.stats_files.s7



## The number of loci caught by each filter.
## ipyrad API location: [assembly].statsfiles.s7_filters

                            total_filters  applied_order  retained_loci
total_prefiltered_loci             420536              0         420536
filtered_by_rm_duplicates           15245          15245         405291
filtered_by_max_indels               2975           2108         403183
filtered_by_max_snps                 3213           2007         401176
filtered_by_max_shared_het           1062            929         400247
filtered_by_min_sample             222616         217547         182700
filtered_by_max_alleles             59779          45794         136906
total_filtered_loci                136906              0         136906


## The number of loci recovered for each Sample.
## ipyrad API location: [assembly].stats_dfs.s7_samples

                          sample_coverage
BP001_dentatum                      59284
ELS161_dentatum                     26644
ELS163_dentat

### Visualize shared data

In [15]:
def getarray(loci, tree, split1=0, split2=0):
    """ parse the loci list and return a presence/absence matrix ordered by 
        the tips on the tree"""
    ## parse the loci file
    #loci = open(locifile).read().split("\n//")[:-1]
    ## order (ladderize) the tree
    tree.ladderize()
    
    ## get tip names
    snames = tree.get_leaf_names()
    
    ## make empty matrix
    lxs = np.zeros((len(snames), len(loci)))
    
    ## fill the matrix
    for loc in xrange(len(loci)):
        for seq in loci[loc].split("\n"):
            if "//" not in seq:
                lxs[snames.index(seq.split()[0][:]), loc] += 1

    return lxs

In [16]:
def countmatrix(lxsabove, lxsbelow, max=0):
    """ fill a matrix with pairwise data sharing
        between each pair of samples. You could put
        in two different 'share' matrices to have
        different results above and below the diagonal.
        Can enter a max value to limit fill along diagonal.
        """
    share = np.zeros((lxsabove.shape[0], 
                      lxsbelow.shape[0]))
    ## fill above
    names = range(lxsabove.shape[0])
    for row in lxsabove:
        for samp1,samp2 in itertools.combinations(names,2):
            shared = lxsabove[samp1, lxsabove[samp2,]>0].sum()
            share[samp1,samp2] = shared
    ## fill below
    for row in lxsbelow:
        for samp2,samp1 in itertools.combinations(names,2):
            shared = lxsabove[samp1, lxsabove[samp2,]>0].sum()
            share[samp1,samp2] = shared
    ## fill diagonal
    if not max:
        for row in range(len(names)):
            share[row,row] = lxsabove[row,].sum()
    else:
        for row in range(len(names)):
            share[row,row] = max
    return share

In [25]:
import ete3
import numpy as np
import itertools
import ipyrad as ip

## reload the ipyrad Assembly if necessary
min4 = ip.load_json("/ysm-gpfs/scratch60/de243/dentatum-ELS-RAD/c90d6f2h5/min4_c90d6f2h5.json")
#analysis_ipyrad/ficus_c85d6f2_min4_s3K.json")

## load the tree that we just inferred
tree = ete3.Tree("((((((ELS203_dentatum,(ELS208_dentatum,dentatum_ELS15)),(NYBG02_dentatum,dentatum_ELS4)),(((ELS450_dentataum,(ELS453_dentatum,ELS452_dentatum)),(ELS585_dentatum,(ELS296_dentatum,ELS292_dentatum))),(((ELS589_dentatum,ELS588_detntatum),((ELS216_dentatum,ELS581_dentatum),ELS213_dentatum)),(ELS193_dentatum,ELS592_dentatum)))),(((ELS599_dentatum,((((ELS646_dentatum,ELS633_dentatum),(((ELS609_dentatum,(ELS275_dentatum,ELS613_dentatum)),(((ELS291_dentatum,ELS272_dentatum),(ELS602_dentatum,ELS597_dentatum)),((ELS281_dentatum,ELS610_dentatum),dentatum_ELS27))),((ELS447_dentatum,(dentatum_ELS52,ELS326_dentatum)),((((ELS623_dentatum,ELS617_dentatum),BP001_dentatum),((ELS442_dentatum,ELS371_dentatum),ELS641_dentatum)),((((MJD001_dentatum,ELS439_dentatum),(ELS421_dentatum,ELS75_dentatum)),(((ELS67_dentatum,dentatum_ELS72),(((ELS382_dentatum,ELS378_dentatum),ELS69_dentatum),(ELS390_dentatum,ELS393_dentatum))),(((ELS550_dentatum,ELS543_dentatum),ELS547_dentatum),ELS533_dentatum))),(((ELS465_dentatum,dentatum_ELS82),(ELS161_dentatum,ELS163_dentatum)),((ELS526_dentatum,(ELS529_dentatum,ELS528_dentatum)),(ELS87_dentatum,ELS650_dentatum)))))))),((ELS170_dentatum,ELS639_dentatum),(ELS640_dentatum,ELS637_dentatum))),(ELS376_dentatum,ELS63_dentatum))),(ELS503_dentatum,ELS512_dentatum)),((ELS577_dentatum,ELS574_dentatum),((ELS559_dentatum,ELS495_dentatum),(ELS570_dentatum,ELS571_dentatum))))),((sulcatum_D9_MEX_003,acutifolium_DRY3_MEX_006),(MJD-76_microcarpum,MJD-64_caudatum))),(MJD-05_molle,MJD-49_ellipticum));")
tree.ladderize()
names = tree.get_leaf_names()

## parse the loci file
locidata = open(min4.outfiles.loci)
loci = locidata.read().split("|\n")[:-1]

  loading Assembly: min4_c90d6f2h5
  from saved path: /ysm-gpfs/scratch60/de243/dentatum-ELS-RAD/c90d6f2h5/min4_c90d6f2h5.json


In [26]:
lxs = getarray(loci, tree)
print lxs.shape
print lxs.sum()

share = countmatrix(lxs, lxs)
print share.shape
print share

(84, 136906)
2567069.0
(84, 84)
[[ 26561.  15744.   9597. ...,  11103.  11111.  11412.]
 [ 15744.  23541.   8400. ...,   9890.   9794.  10021.]
 [  9597.   8400.  24697. ...,   9238.   9221.   9246.]
 ..., 
 [ 11103.   9890.   9238. ...,  32244.  14465.  14826.]
 [ 11111.   9794.   9221. ...,  14465.  31961.  15087.]
 [ 11412.  10021.   9246. ...,  14826.  15087.  32984.]]


In [27]:
import toyplot
import toyplot.html

## get a good color map
colormap = toyplot.color.LinearMap(toyplot.color.brewer.palette("Spectral"), 
                                   domain_min=share.min(), 
                                   domain_max=share.max())

In [30]:
def plotshare(share, names):
    
    ## set up canvas
    canvas = toyplot.Canvas(width=900, height=700)
    
    ## order for data
    table = canvas.matrix((share, colormap), 
                          bounds=(50, 600, 50, 600), 
                          step=5, tshow=False, lshow=False)
    
    ## put a box around it
    table.body.grid.vlines[...,[0,-1]] = 'single'
    table.body.grid.hlines[[0,-1],...] = 'single'
    
    ## make floater for grid
    for i,j in itertools.product(range(len(share)), repeat=2):
        table.body.cell(i,j).title='%s, %s : %s' % (names[i],
                                                    names[j],
                                                    int(share[i,j]))
    ## canvas for barplot
    axes = canvas.cartesian(bounds=(665, 800, 90, 560))
    ## make floater for barplot
    zf = zip(names[::-1], share.diagonal()[::-1])
    barfloater = ["%s: %s" % (i,int(j)) for i,j in zf]
    
    ## create barplot
    axes.bars(share.diagonal()[::-1], 
              along="y",
              title = barfloater)
        
    ## Hide yspine, move labels to the left, 
    ## use taxon names, rotate angle, align.
    axes.y.spine.show = False
    axes.y.ticks.labels.offset = 0
    axes.y.ticks.locator = toyplot.locator.Explicit(range(len(names)),
                                           labels=names[::-1])
    axes.y.ticks.labels.angle = -90
    axes.y.ticks.labels.style = {"baseline-shift":0,
                                 "text-anchor":"end",
                                 "font-size":"8px"} 
        
    ## Rotate xlabels, align with ticks, 
    ## change to thousands, move up on canvas,
    ## show ticks, and hide popup coordinates
    axes.x.ticks.labels.angle = 90
    axes.x.ticks.labels.offset = 20
    axes.x.ticks.locator = toyplot.locator.Explicit(
        range(0, 40000, 5000), 
        ["{}K".format(i) for i in range(0, 40, 5)])
    axes.x.ticks.labels.style = {"baseline-shift":0, 
                                 "text-anchor":"end", 
                                 "-toyplot-anchor-shift":"15px"}
    axes.x.ticks.show = True
    
    ## add labels
    label_style = {"font-size": "16px", "font-weight": "bold"}
    canvas.text(300, 60, "(a) Matrix of shared RAD loci", style=label_style)
    canvas.text(700, 60, "(b) N RAD loci per sample", style=label_style)

    
    ## add colormap
    #canvas.color_scale(colormap)
    
    ## save as html
    #toyplot.html.render(canvas, "c85d5f2_min4_s3k_datasharing.html")
    
    
plotshare(share, names)

MJD-05_molle, MJD-05_molle : 26561 MJD-05_molle, MJD-49_ellipticum : 15744 MJD-05_molle, sulcatum_D9_MEX_003 : 9597 MJD-05_molle, acutifolium_DRY3_MEX_006 : 11353 MJD-05_molle, MJD-76_microcarpum : 12072 MJD-05_molle, MJD-64_caudatum : 11281 MJD-05_molle, NYBG02_dentatum : 3450 MJD-05_molle, dentatum_ELS4 : 14453 MJD-05_molle, ELS203_dentatum : 8323 MJD-05_molle, ELS208_dentatum : 3435 MJD-05_molle, dentatum_ELS15 : 11726 MJD-05_molle, ELS450_dentataum : 7162 MJD-05_molle, ELS453_dentatum : 9388 MJD-05_molle, ELS452_dentatum : 11290 MJD-05_molle, ELS585_dentatum : 7848 MJD-05_molle, ELS296_dentatum : 9506 MJD-05_molle, ELS292_dentatum : 11771 MJD-05_molle, ELS193_dentatum : 9030 MJD-05_molle, ELS592_dentatum : 12067 MJD-05_molle, ELS589_dentatum : 10002 MJD-05_molle, ELS588_detntatum : 9307 MJD-05_molle, ELS213_dentatum : 6324 MJD-05_molle, ELS216_dentatum : 11075 MJD-05_molle, ELS581_dentatum : 3236 MJD-05_molle, ELS577_dentatum : 8970 MJD-05_molle, ELS574_dentatum : 9935 MJD-05_molle, ELS559_dentatum : 9868 MJD-05_molle, ELS495_dentatum : 7164 MJD-05_molle, ELS570_dentatum : 9317 MJD-05_molle, ELS571_dentatum : 9601 MJD-05_molle, ELS503_dentatum : 10599 MJD-05_molle, ELS512_dentatum : 11574 MJD-05_molle, ELS599_dentatum : 8273 MJD-05_molle, ELS376_dentatum : 9037 MJD-05_molle, ELS63_dentatum : 8328 MJD-05_molle, ELS170_dentatum : 9154 MJD-05_molle, ELS639_dentatum : 8407 MJD-05_molle, ELS640_dentatum : 9599 MJD-05_molle, ELS637_dentatum : 9616 MJD-05_molle, ELS646_dentatum : 10822 MJD-05_molle, ELS633_dentatum : 9157 MJD-05_molle, ELS609_dentatum : 13152 MJD-05_molle, ELS275_dentatum : 9990 MJD-05_molle, ELS613_dentatum : 11139 MJD-05_molle, dentatum_ELS27 : 15537 MJD-05_molle, ELS281_dentatum : 12014 MJD-05_molle, ELS610_dentatum : 10509 MJD-05_molle, ELS291_dentatum : 11254 MJD-05_molle, ELS272_dentatum : 10492 MJD-05_molle, ELS602_dentatum : 11449 MJD-05_molle, ELS597_dentatum : 13723 MJD-05_molle, ELS447_dentatum : 10843 MJD-05_molle, dentatum_ELS52 : 12293 MJD-05_molle, ELS326_dentatum : 12085 MJD-05_molle, BP001_dentatum : 17823 MJD-05_molle, ELS623_dentatum : 10137 MJD-05_molle, ELS617_dentatum : 10808 MJD-05_molle, ELS641_dentatum : 13065 MJD-05_molle, ELS442_dentatum : 10239 MJD-05_molle, ELS371_dentatum : 10071 MJD-05_molle, ELS465_dentatum : 11505 MJD-05_molle, dentatum_ELS82 : 14290 MJD-05_molle, ELS161_dentatum : 9439 MJD-05_molle, ELS163_dentatum : 12394 MJD-05_molle, ELS87_dentatum : 9738 MJD-05_molle, ELS650_dentatum : 18114 MJD-05_molle, ELS526_dentatum : 10662 MJD-05_molle, ELS529_dentatum : 10805 MJD-05_molle, ELS528_dentatum : 10338 MJD-05_molle, MJD001_dentatum : 16293 MJD-05_molle, ELS439_dentatum : 12903 MJD-05_molle, ELS421_dentatum : 11457 MJD-05_molle, ELS75_dentatum : 12395 MJD-05_molle, ELS533_dentatum : 10437 MJD-05_molle, ELS547_dentatum : 9971 MJD-05_molle, ELS550_dentatum : 9850 MJD-05_molle, ELS543_dentatum : 9644 MJD-05_molle, ELS67_dentatum : 11702 MJD-05_molle, dentatum_ELS72 : 11051 MJD-05_molle, ELS390_dentatum : 9936 MJD-05_molle, ELS393_dentatum : 12298 MJD-05_molle, ELS69_dentatum : 11103 MJD-05_molle, ELS382_dentatum : 11111 MJD-05_molle, ELS378_dentatum : 11412 MJD-49_ellipticum, MJD-05_molle : 15744 MJD-49_ellipticum, MJD-49_ellipticum : 23541 MJD-49_ellipticum, sulcatum_D9_MEX_003 : 8400 MJD-49_ellipticum, acutifolium_DRY3_MEX_006 : 10078 MJD-49_ellipticum, MJD-76_microcarpum : 10653 MJD-49_ellipticum, MJD-64_caudatum : 9963 MJD-49_ellipticum, NYBG02_dentatum : 3182 MJD-49_ellipticum, dentatum_ELS4 : 12704 MJD-49_ellipticum, ELS203_dentatum : 7372 MJD-49_ellipticum, ELS208_dentatum : 3087 MJD-49_ellipticum, dentatum_ELS15 : 10437 MJD-49_ellipticum, ELS450_dentataum : 6406 MJD-49_ellipticum, ELS453_dentatum : 8345 MJD-49_ellipticum, ELS452_dentatum : 9928 MJD-49_ellipticum, ELS585_dentatum : 6972 MJD-49_ellipticum, ELS296_dentatum : 8284 MJD-49_ellipticum, ELS292_dentatum : 10348 MJD-49_ellipticum, ELS193_dentatum : 7890 MJD-49_ellipticum, ELS592_dentatum : 10775 MJD-49_ellipticum,

# Analyses

In [67]:
## in case you need to re-load the data set
min10 = ip.load_json("/fastscratch/de243/ELS-dent/c90d6f2h5/min10_c90d6f2h5.json")

  loading Assembly: min10_c90d6f2h5
  from saved path: /fastscratch/de243/ELS-dent/c90d6f2h5/min10_c90d6f2h5.json


### RAxML

In [68]:
## make raxml dir
RAXDIR = os.path.join(os.curdir, "analysis_raxml")
RAXDIR = os.path.realpath(RAXDIR)
if not os.path.exists(RAXDIR):
    os.mkdir(RAXDIR)

## get path to louise' system-wide raxml
LORAX = "/home2/de243/miniconda2/bin/raxmlHPC-PTHREADS"

## set outgroup string... shoot I forgot outgroups.
#OUT = ",".join([])
    
## set up arguments
cmd = [LORAX,
        "-f", "a", 
        "-m", "GTRGAMMA", 
        "-N", "100", 
        "-T", "32", 
        "-x", "12345", 
        "-p", "54321",
        "-w", RAXDIR,
        "-n", min10.name, 
        "-s", min10.outfiles.loci[:-5]+'.phy']
        #"-o", OUT, 
        
## send to process as non-blocking job
raxjob = sps.Popen(cmd, stderr=sps.PIPE, stdout=sps.PIPE)

In [85]:
## ask whether it's done
## ask if it's still running in background
if raxjob.poll():
    print raxjob.returncode
else:
    tail = ! tail $RAXDIR/*info*
    print "still running: \ntail:", tail[-1]

still running: 
tail: Bootstrap[6]: Time 14229.471893 seconds, bootstrap likelihood -13844919.338222, best rearrangement setting 6


### PCA view 

In [29]:
%%bash
## uncomment to install pre-reqs for plotting 
#conda install matplotlib -y 
#conda install seaborn -y
#conda install -c bioconda pyvcf -y
#pip install vcfnp
#pip install scikit-allel
#pip install toyplot

In [43]:
import vcf
import vcfnp
import allel
import toyplot

In [ ]:
## unzip the VCF file (why doesn't this support compression?)
! gunzip $min4.outfiles.vcf

In [61]:
min4.outfiles.vcf = min4.outfiles.vcf[:-3]
print min4.outfiles.vcf

/fastscratch/de243/ELS-dent/c90d6f2h5/min4_c90d6f2h5_outfiles/min4_c90d6f2h5.vcf


In [62]:
v = vcfnp.variants(data.outfiles.vcf, cache=True).view(np.recarray)

[vcfnp] 2016-11-03 13:34:55.656136 :: caching is enabled
[vcfnp] 2016-11-03 13:34:55.656859 :: cache file available
[vcfnp] 2016-11-03 13:34:55.657238 :: loading from cache file /fastscratch/de243/ELS-dent/c90d6f2h5/min4_c90d6f2h5_outfiles/min4_c90d6f2h5.vcf.vcfnp_cache/variants.npy


In [63]:
# print some simple variant metrics
print('found %s variants (%s SNPs)' % (v.size, np.count_nonzero(v.is_snp)))
print('QUAL mean (std): %s (%s)' % (np.mean(v.QUAL), np.std(v.QUAL)))

found 11030413 variants (628578 SNPs)
QUAL mean (std): 13.0 (9.53674e-07)


In [30]:
min4.outfiles.vcf

## unzip vcf If it's gzipped then unzip it (only applies to ipyrad i think)
cmd = "gunzip {}".format(f)
cmd
            f = f.split(".gz")[0]
            
        ## Remove all but biallelic (for ipyrad this also removes all the monomorphic)
        outfile = f.split(".vcf")[0]+"biallelic.vcf"
        cmd = "{}vcftools --vcf {} --min-alleles 2 --max-alleles 2 --recode --out {}" \
            .format(DDOCENT_DIR, f, outfile)
        print(cmd)
        !$cmd
        
        ## update the vcf_dict
        vcf_dict[k] = outfile
    else:
        print("not found - {}".format(f))

'/fastscratch/de243/ELS-dent/c90d6f2h5/min4_c90d6f2h5_outfiles/min4_c90d6f2h5.vcf.gz'

### Allele/ploidy variation